# classification

In [2]:
import numpy as np 
import matplotlib.pyplot as plt 

import cv2 as cv 
from PIL import Image 

import time

from pycoral.adapters import classify, common
from pycoral.utils.edgetpu import make_interpreter
from pycoral.utils.dataset import read_label_file

from tflite_runtime.interpreter import Interpreter
from tflite_runtime.interpreter import load_delegate

from reachy_sdk import ReachySDK

from reachy_sdk.trajectory import goto
from reachy_sdk.trajectory.interpolation import InterpolationMode

import argparse

from PIL import ImageDraw

from pycoral.adapters import detect
from pycoral.utils.dataset import read_label_file
from pycoral.utils.edgetpu import make_interpreter

In [3]:
reachy = ReachySDK('localhost')

In [3]:
path_model = '/home/reachy/Desktop/tflite/output_tflite_graph_edgetpu.tflite'
path_label = '/home/reachy/Desktop/tflite/label.txt'

In [ ]:
path_model = '/home/reachy/Reachy_Nautilus/Notebook/tflite/model.tflite'
path_label = '/home/reachy/dev/reachy-tictactoe/reachy_tictactoe/models/ttt-boxes.txt'


In [4]:
interpreter = make_interpreter(path_model)
interpreter.allocate_tensors()

labels = read_label_file(path_label)
print(labels)
size = common.input_size(interpreter)

{0: 'empty', 1: 'cylinder', 2: 'cube'}


### image

In [9]:
#img = Image(filename='/home/reachy/Reachy_Nautilus/images/image20.png')
img = Image.open('/home/reachy/Reachy_Nautilus/images/image12.png')
a = np.asarray(img)
pil_img = Image.fromarray(a).convert('RGB').resize(size, Image.ANTIALIAS)

common.set_input(interpreter, pil_img)
interpreter.invoke()
result = classify.get_classes(interpreter, top_k=1, score_threshold=0.8)
print(f' resultat ={result}')
label = labels.get(result[0].id)     

 resultat =[Class(id=11, score=1.0116534)]


In [7]:
help(classify.get_classes)

Help on function get_classes in module pycoral.adapters.classify:

get_classes(interpreter, top_k=inf, score_threshold=-inf)
    Gets results from a classification model as a list of ordered classes.
    
    Args:
      interpreter: The ``tf.lite.Interpreter`` to query for results.
      top_k (int): The number of top results to return.
      score_threshold (float): The score threshold for results. All returned
        results have a score greater-than-or-equal-to this value.
    
    Returns:
      A list of :obj:`Class` objects representing the classification results,
      ordered by scores.



### video

In [4]:
board_cases = np.array((
    ((86, 169, 388, 464), #Coordinates first board cases (top-left corner) (Xbl, Xbr, Ytr, Ybr)
     (169, 261, 388, 464), #Coordinates second board cases
     (261, 354, 388, 464),),

    ((77, 167, 446, 547),
     (167, 268, 446, 547),
     (268, 365, 446, 547),),

    ((70, 164, 551, 629),
     (164, 270, 551, 629),
     (270, 375, 551, 629),),
))

# left, right, top, bottom
board_rect = np.array((
    67, 375, 374, 629,
))

In [5]:
im = reachy.right_camera.wait_for_new_frame()
height, width = np.shape(im)[:2]


In [9]:
video = cv.VideoWriter('tuto_classification300.avi',cv.VideoWriter_fourcc('M','J','P','G'), 10, (width,height))

reachy.turn_on('head')
reachy.head.look_at(x=1, y=0, z=0, duration=1.5)  
time.sleep(1.5)

reachy.head.look_at(0.5, 0, -0.5, duration=1.25)
time.sleep(1.25)
start = time.time()

fps_count = 0

while time.time() - start < 15:
    
    im = reachy.right_camera.wait_for_new_frame()
    out = im.copy()

    for row in range(3):
        for col in range(3):
            lx, rx, ly, ry = board_cases[row, col]
            pil_img = Image.fromarray(out[ly:ry, lx:rx]).convert('RGB').resize(size, Image.ANTIALIAS)
            pil_img.save(f'/home/reachy/Desktop/image{row}{col}.png')
            common.set_input(interpreter, pil_img)
            interpreter.invoke()
            result = classify.get_classes(interpreter, top_k=3, score_threshold=0.8)
            print(f' resultat ={result}')
            if result :
                label = labels.get(result[0].id)
                cv.putText(out, label, (int((lx+rx)/2)-30, int((ly+ry)/2)), cv.FONT_HERSHEY_SIMPLEX, 0.75, (255,0,0), 2)
                cv.rectangle(out, (lx, ly), (rx, ry), (0, 255, 0), 5)
    video.write(out)        
            
video.release()

reachy.head.look_at(0.5, 0, 0, duration=1)
time.sleep(0.2)

 resultat =[Class(id=6, score=1.0015632), Class(id=3, score=0.9669991), Class(id=7, score=0.8990731)]
 resultat =[Class(id=18, score=1.1149503), Class(id=15, score=1.0585748), Class(id=11, score=1.01602)]
 resultat =[Class(id=18, score=1.1261232), Class(id=15, score=1.0520699), Class(id=10, score=1.0171156)]
 resultat =[Class(id=18, score=1.1261232), Class(id=10, score=1.1161618), Class(id=15, score=1.0520699)]
 resultat =[Class(id=31, score=1.1118968), Class(id=18, score=1.1016064), Class(id=14, score=1.050638)]
 resultat =[Class(id=31, score=1.1118968), Class(id=18, score=1.0530658), Class(id=27, score=1.0212201)]
 resultat =[Class(id=31, score=1.1118968), Class(id=14, score=1.1068835), Class(id=18, score=1.0530658)]
 resultat =[Class(id=31, score=1.1118968), Class(id=18, score=1.0530658), Class(id=15, score=1.0527434)]
 resultat =[Class(id=31, score=1.1118968), Class(id=26, score=1.0767032), Class(id=23, score=1.0620579)]
 resultat =[Class(id=31, score=1.1118968), Class(id=26, score

 resultat =[Class(id=22, score=1.1294824), Class(id=34, score=1.0813409), Class(id=19, score=1.0527434)]
 resultat =[Class(id=34, score=1.0813409), Class(id=30, score=1.0812974), Class(id=22, score=1.0442774)]
 resultat =[Class(id=34, score=1.0813409), Class(id=30, score=1.0812974), Class(id=22, score=1.0442774)]
 resultat =[Class(id=34, score=1.0813409), Class(id=30, score=1.0812974), Class(id=22, score=1.0442774)]
 resultat =[Class(id=10, score=1.0876892), Class(id=34, score=1.0813409), Class(id=30, score=1.0812974)]
 resultat =[Class(id=14, score=1.1318194), Class(id=34, score=1.0813409), Class(id=30, score=1.0812974)]
 resultat =[Class(id=27, score=1.0973003), Class(id=34, score=1.0813409), Class(id=30, score=1.0812974)]
 resultat =[Class(id=27, score=1.0973003), Class(id=34, score=1.0813409), Class(id=30, score=1.0812974)]
 resultat =[Class(id=27, score=1.0973003), Class(id=18, score=1.0966257), Class(id=34, score=1.0813409)]
 resultat =[Class(id=15, score=1.1298553), Class(id=27,

 resultat =[Class(id=22, score=1.1164669), Class(id=19, score=1.072461), Class(id=31, score=1.0692008)]
 resultat =[Class(id=22, score=1.1164669), Class(id=31, score=1.0692008), Class(id=18, score=1.0499408)]
 resultat =[Class(id=22, score=1.1164669), Class(id=14, score=1.1163346), Class(id=31, score=1.0692008)]
 resultat =[Class(id=22, score=1.1164669), Class(id=14, score=1.1163346), Class(id=31, score=1.0692008)]
 resultat =[Class(id=22, score=1.1164669), Class(id=31, score=1.0692008), Class(id=15, score=1.0589691)]
 resultat =[Class(id=14, score=1.1211865), Class(id=22, score=1.1164669), Class(id=31, score=1.0692008)]
 resultat =[Class(id=35, score=1.0460916), Class(id=39, score=1.0393972), Class(id=31, score=1.0358169)]
 resultat =[Class(id=14, score=1.0470176), Class(id=35, score=1.0460916), Class(id=39, score=1.0393972)]
 resultat =[Class(id=14, score=1.1076118), Class(id=18, score=1.0964121), Class(id=35, score=1.0460916)]
 resultat =[Class(id=14, score=1.1076118), Class(id=18, 

### avec tflite_runtime : 

In [5]:
def load_labels(path):
    with open(path, 'r') as f:
        return {i: line.strip() for i, line in enumerate(f.readlines())}


def set_input_tensor(interpreter, image):
    tensor_index = interpreter.get_input_details()[0]['index']
    input_tensor = interpreter.tensor(tensor_index)()[0]
    input_tensor[:, :] = image

def classify_image(interpreter, image, top_k=3):
    """Returns a sorted array of classification results."""
    set_input_tensor(interpreter, image)
    interpreter.invoke()
    output_details = interpreter.get_output_details()[0]
    output = np.squeeze(interpreter.get_tensor(output_details['index']))
    print(f'output = {output}')

    # If the model is quantized (uint8 data), then dequantize the results
    if output_details['dtype'] == np.uint8:
        print('passee')
        scale, zero_point = output_details['quantization']
        output = scale * (output - zero_point)

    ordered = np.argpartition(-output, top_k)
    return [(i, output[i]) for i in ordered[:top_k]]

In [6]:
labels = load_labels(path_label)

interpreter = Interpreter(path_model)
#interpreter = Interpreter(path_model, experimental_delegates=[load_delegate('libedgetpu.so.1.0')])
interpreter.allocate_tensors()
_, height, width, _ = interpreter.get_input_details()[0]['shape']

reachy.turn_on('head')
reachy.head.look_at(x=1, y=0, z=0, duration=1.5)  
time.sleep(1.5)

reachy.head.look_at(0.5, 0, -0.5, duration=1.25)
time.sleep(1.25)
start = time.time()

fps_count = 0
nb=0

while time.time() - start < 15:
    
    im = reachy.right_camera.wait_for_new_frame()
    print(type(im))
    out = im.copy()
    pil_img = Image.fromarray(out).convert('RGB').resize((width, height),Image.ANTIALIAS)
    #pil_img = Image.fromarray(out[ly:ry, lx:rx]).convert('RGB').resize(size, Image.ANTIALIAS)
    pil_img.save(f'/home/reachy/Desktop/image{nb}.png')
    #common.set_input(interpreter, pil_img)
    #interpreter.invoke()
    results = classify_image(interpreter, pil_img)
    label_id, prob = results[0]
    #result = classify.get_classes(interpreter, top_k=3, score_threshold=0.8)
    print(f' resultat ={results}')
    #print(f' resultat ={labels[label_id]}')
    cv.putText(out,labels[label_id], cv.FONT_HERSHEY_SIMPLEX, 0.75, (255,0,0), 2)
    nb = nb+1

reachy.head.look_at(0.5, 0, 0, duration=1)
time.sleep(0.2)

RuntimeError: Encountered unresolved custom op: edgetpu-custom-op.Node number 0 (edgetpu-custom-op) failed to prepare.


### avec code exemple pycoral : 

In [7]:
def draw_objects(draw, objs, labels):
#Draws the bounding box and label for each object.
#for obj in objs:
    obj = objs[0]
    bbox = obj.bbox
    draw.rectangle([(bbox.xmin, bbox.ymin), (bbox.xmax, bbox.ymax)],
                   outline='red')
    draw.text((bbox.xmin + 10, bbox.ymin + 10),
              '%s\n%.2f' % (labels.get(obj.id, obj.id), obj.score),
              fill='red')

pour les images : 

In [45]:
path_model = '/home/reachy/Desktop/tflite/output_tflite_graph_edgetpu.tflite'
path_label = '/home/reachy/Desktop/tflite/label.txt'

labels = read_label_file(path_label) if path_label else {}
interpreter = make_interpreter(path_model)
interpreter.allocate_tensors()

image = Image.open('/home/reachy/Desktop/tflite/image21.png')
_, scale = common.set_resized_input(interpreter, image.size, lambda size: image.resize(size, Image.ANTIALIAS))


args_count = 5
args_threshold = 0.5

for _ in range(args_count):
    start = time.perf_counter()
    interpreter.invoke()
    inference_time = time.perf_counter() - start
    objs = detect.get_objects(interpreter, args_threshold, scale)
    print('%.2f ms' % (inference_time * 1000))

print('-------RESULTS--------')
if not objs:
    print('No objects detected')

#for obj in objs:
obj = objs[0]
print(labels.get(obj.id, obj.id))
print('  id:    ', obj.id)
print('  score: ', obj.score)
print('  bbox:  ', obj.bbox)

image = image.convert('RGB')
draw_objects(ImageDraw.Draw(image), objs, labels)
image.save('image.png')
image.show()


18.48 ms
6.17 ms
6.77 ms
9.71 ms
6.25 ms
-------RESULTS--------
cylinder
  id:     2
  score:  0.875
  bbox:   BBox(xmin=39, ymin=-4, xmax=178, ymax=137)


In [6]:
reachy.turn_off('reachy')

pour la vidéo : 

In [5]:
im = reachy.right_camera.wait_for_new_frame()
height, width = np.shape(im)[:2]

In [8]:
video = cv.VideoWriter('tuto_classification400.avi',cv.VideoWriter_fourcc('M','J','P','G'), 5,(width,height))

reachy.turn_on('head')
reachy.head.look_at(x=1, y=0, z=0, duration=1.5)  
time.sleep(1.5)

reachy.head.look_at(x=0.75, y=0, z=-0.4, duration=1.5) 
path_model = '/home/reachy/Desktop/tflite/output_tflite_graph_edgetpu.tflite'
path_label = '/home/reachy/Desktop/tflite/label.txt'

labels = read_label_file(path_label) if path_label else {}
interpreter = make_interpreter(path_model)
interpreter.allocate_tensors()

start = time.time()
fps_count = 0
nb = 1
while time.time() - start < 15:
    
    print(f'temps : {time.time() - start}')
    im = reachy.right_camera.wait_for_new_frame()
    out = im.copy()

    for row in range(3):
        for col in range(3):
            lx, rx, ly, ry = board_cases[row, col]
            
            image = Image.fromarray(out[ly:ry, lx:rx])
            image.save(f'/home/reachy/Desktop/image{nb}.png')
            nb = nb+1
            _, scale = common.set_resized_input(interpreter, image.size, lambda size: image.resize(size, Image.ANTIALIAS))

            args_count = 1
            args_threshold = 0.5

            for _ in range(args_count):
                start2 = time.perf_counter()
                interpreter.invoke()
                inference_time = time.perf_counter() - start2
                objs = detect.get_objects(interpreter, args_threshold, scale)
                print('%.2f ms' % (inference_time * 1000))

            print('-------RESULTS--------')
            if not objs:
                print('No objects detected')

            #for obj in objs:
            obj = objs[0]
            print(labels.get(obj.id, obj.id))
            print('  id:    ', obj.id)
            print('  score: ', obj.score)
            print('  bbox:  ', obj.bbox)
            label = 'pouet'
            if (obj.id == 0):
                label = 'empty'
            if (obj.id == 1):
                label = 'cube'
            if (obj.id == 2):
                label = 'cylinder'

            #image = image.convert('RGB')
            #draw_objects(ImageDraw.Draw(image), objs, labels)
            #image.save('image.png')
            #image.show()
            
            #label = labels.get(result[0].id)
            cv.putText(out, label, (int((lx+rx)/2)-30, int((ly+ry)/2)), cv.FONT_HERSHEY_SIMPLEX, 0.75, (255,0,0), 2)
            cv.rectangle(out, (lx, ly), (rx, ry), (0, 255, 0), 5)
            #time.sleep(5)
    video.write(out)        
            
video.release()

reachy.head.look_at(0.5, 0, 0, duration=1)
time.sleep(0.2)

temps : 0.00041413307189941406
26.84 ms
-------RESULTS--------
cube
  id:     1
  score:  0.9296875
  bbox:   BBox(xmin=0, ymin=3, xmax=78, ymax=77)
6.20 ms
-------RESULTS--------
cube
  id:     1
  score:  0.65625
  bbox:   BBox(xmin=-1, ymin=13, xmax=85, ymax=89)
6.14 ms
-------RESULTS--------
cube
  id:     1
  score:  0.65625
  bbox:   BBox(xmin=0, ymin=63, xmax=84, ymax=95)
6.16 ms
-------RESULTS--------
cube
  id:     1
  score:  0.875
  bbox:   BBox(xmin=14, ymin=12, xmax=86, ymax=83)
6.68 ms
-------RESULTS--------
cube
  id:     1
  score:  0.96484375
  bbox:   BBox(xmin=7, ymin=15, xmax=89, ymax=74)
9.33 ms
-------RESULTS--------
cube
  id:     1
  score:  0.9296875
  bbox:   BBox(xmin=-3, ymin=14, xmax=80, ymax=76)
11.17 ms
-------RESULTS--------
cube
  id:     1
  score:  0.9296875
  bbox:   BBox(xmin=1, ymin=3, xmax=92, ymax=78)
8.60 ms
-------RESULTS--------
cube
  id:     1
  score:  0.9296875
  bbox:   BBox(xmin=-2, ymin=6, xmax=97, ymax=82)
11.07 ms
-------RESULTS------

10.35 ms
-------RESULTS--------
empty
  id:     0
  score:  0.78515625
  bbox:   BBox(xmin=2, ymin=47, xmax=87, ymax=102)
7.25 ms
-------RESULTS--------
empty
  id:     0
  score:  0.78515625
  bbox:   BBox(xmin=2, ymin=30, xmax=88, ymax=95)
7.46 ms
-------RESULTS--------
cube
  id:     1
  score:  0.78515625
  bbox:   BBox(xmin=28, ymin=20, xmax=95, ymax=96)
7.34 ms
-------RESULTS--------
cube
  id:     1
  score:  0.78515625
  bbox:   BBox(xmin=30, ymin=21, xmax=95, ymax=95)
temps : 4.236771821975708
6.68 ms
-------RESULTS--------
empty
  id:     0
  score:  0.78515625
  bbox:   BBox(xmin=6, ymin=29, xmax=80, ymax=82)
31.24 ms
-------RESULTS--------
cylinder
  id:     2
  score:  0.65625
  bbox:   BBox(xmin=17, ymin=18, xmax=78, ymax=79)
7.75 ms
-------RESULTS--------
empty
  id:     0
  score:  0.65625
  bbox:   BBox(xmin=0, ymin=26, xmax=82, ymax=91)
6.25 ms
-------RESULTS--------
empty
  id:     0
  score:  0.875
  bbox:   BBox(xmin=8, ymin=48, xmax=89, ymax=103)
41.54 ms
-------R

6.25 ms
-------RESULTS--------
empty
  id:     0
  score:  0.78515625
  bbox:   BBox(xmin=7, ymin=31, xmax=80, ymax=83)
23.19 ms
-------RESULTS--------
cylinder
  id:     2
  score:  0.875
  bbox:   BBox(xmin=17, ymin=16, xmax=80, ymax=77)
6.75 ms
-------RESULTS--------
empty
  id:     0
  score:  0.5
  bbox:   BBox(xmin=3, ymin=22, xmax=80, ymax=86)
39.28 ms
-------RESULTS--------
empty
  id:     0
  score:  0.9296875
  bbox:   BBox(xmin=8, ymin=44, xmax=90, ymax=102)
9.51 ms
-------RESULTS--------
cube
  id:     1
  score:  0.96484375
  bbox:   BBox(xmin=12, ymin=38, xmax=79, ymax=99)
30.98 ms
-------RESULTS--------
empty
  id:     0
  score:  0.78515625
  bbox:   BBox(xmin=2, ymin=49, xmax=87, ymax=100)
16.57 ms
-------RESULTS--------
empty
  id:     0
  score:  0.78515625
  bbox:   BBox(xmin=4, ymin=29, xmax=87, ymax=94)
7.38 ms
-------RESULTS--------
empty
  id:     0
  score:  0.78515625
  bbox:   BBox(xmin=1, ymin=31, xmax=96, ymax=105)
6.25 ms
-------RESULTS--------
empty
  id:

8.54 ms
-------RESULTS--------
empty
  id:     0
  score:  0.78515625
  bbox:   BBox(xmin=6, ymin=29, xmax=90, ymax=94)
7.58 ms
-------RESULTS--------
empty
  id:     0
  score:  0.78515625
  bbox:   BBox(xmin=0, ymin=28, xmax=93, ymax=104)
9.86 ms
-------RESULTS--------
cube
  id:     1
  score:  0.78515625
  bbox:   BBox(xmin=26, ymin=22, xmax=96, ymax=88)
temps : 11.975196838378906
23.70 ms
-------RESULTS--------
empty
  id:     0
  score:  0.65625
  bbox:   BBox(xmin=6, ymin=31, xmax=77, ymax=82)
12.09 ms
-------RESULTS--------
cylinder
  id:     2
  score:  0.9296875
  bbox:   BBox(xmin=16, ymin=16, xmax=80, ymax=77)
8.48 ms
-------RESULTS--------
empty
  id:     0
  score:  0.65625
  bbox:   BBox(xmin=1, ymin=25, xmax=81, ymax=91)
18.52 ms
-------RESULTS--------
empty
  id:     0
  score:  0.9296875
  bbox:   BBox(xmin=10, ymin=41, xmax=88, ymax=103)
13.75 ms
-------RESULTS--------
cube
  id:     1
  score:  0.96484375
  bbox:   BBox(xmin=11, ymin=39, xmax=82, ymax=99)
7.21 ms
--